In [59]:
def debase64(data:str)->bytes:
    table='ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'
    decode = bytearray()
    s = 0
    for e in range(4,len(data)+1,4):
        tmp = 0
       ## print("s={},e={}".format(s,e))
        for i,c in enumerate(data[s:e]):
            if c != "=":
                ##print(c,i)
                tmp += table.index(c) << 24 - (i+1)*6
            else:
                tmp += 0 << 24 - (i+1)*6
        decode.extend(tmp.to_bytes(3,'big'))
        s +=4
    return bytes(decode.rstrip(b'\x00'))

In [60]:
e = 'YWJjZA=='
debase64(e)

b'abcd'

*参数检查装饰器

In [1]:
def add(x:int,y:int) ->int:
    return x + y

In [2]:
import inspect
add(3,4)
parms = inspect.signature(add).parameters
print(parms.keys())
lst=list(parms.values())[0]
lst
#for i in lst:
#    print(i)
#list(parms.values())[0]
lst

odict_keys(['x', 'y'])


<Parameter "x:int">

In [66]:
import functools
functools.wraps
functools.partial
#partial 函数用来固定函数中的一个或多个参数
new_add = functools.partial(add,y=3)
new_add(5)

8

In [67]:
import inspect
import functools

def para_check(fn):
    @functools.wraps(fn)
    def warp(*args,**kwargs):
        sig = inspect.signature(fn)
        parms = sig.parameters
        for k,v in kwargs.items():
            if not isinstance(v,parms[k].annotation) and parms[k].annotation != inspect._empty:
                raise TypeError('parameter {} type required {},but type {}'.format(k,parms[k].annotation,type(v)))
        for i,arg in enumerate(args):
            para = list(parms.values())[i]
            if not isinstance(arg,para.annotation) and para.annotation != inspect._empty:
                raise TypeError('parameter {} type required {},but type {}'.format(arg,para.annotation,type(arg)))
        return fn(*args,**kwargs)
    return warp

In [68]:
@para_check
def add(x,y:int) ->int:
    return x + y

In [69]:
add(1,2)

3

## 带过期的LRU 缓存器装饰器：
*FIFO:先进先出

*LFU：Least Frequently Used 计数

*LRU:Least recently used  计时

*思路：Value 里加一个最后一次使用的时间戳

In [71]:
import inspect
import functools
import datetime

def cache(maxsize=128,expire=0):
    def make_key(fn,args,kwargs):
        ret = []
        names = set()
        params = inspect.signature(fn).parameters
        keys = list(args.keys())
        for i,arg in enumerate(args):
            ret.append(keys[i],arg)
            names.add(keys[i])
        ret.extend(kwargs.itmes())
        names.update(kwargs.keys())
        for k,v in params.items():
            if k not in names:
                ret.append((k,v.default))
        ret.sort(key=lambda x:x[0])
        return '&'.join(['{}={}'.format(name,arg) for name,arg in ret])

##求最大公约数算法
```
s1 = 'aaaba'
s2 = 'aabaa'
1, 1, 1, 0, 1
1, 2, 2, 0, 1 
0, 0, 0, 3, 0
1, 1, 1, 0, 4 
1, 2, 2, 0, 1
```

In [74]:
s1 = 'aaaba'
s2 = 'aabaa'
def lcs(s1,s2):
    ml = 0
    mi = 0
    db = []
    for a,x in enumerate(s1):
        db.append([])
        for b,y in enumerate(s2):
            if x == y:
                if a == 0 or b== 0:
                    db[a].append(1)
                else:
                    db[a].append(db[a-1][b-1]+1)
                if db[a][b] > ml:
                    ml = db[a][b]
                    mi = a+1-ml
            else:
                db[a].append(0)
        print(db)
    return s1[mi:mi+ml]

In [75]:
lcs(s2,s1)

[[1, 1, 1, 0, 1]]
[[1, 1, 1, 0, 1], [1, 2, 2, 0, 1]]
[[1, 1, 1, 0, 1], [1, 2, 2, 0, 1], [0, 0, 0, 3, 0]]
[[1, 1, 1, 0, 1], [1, 2, 2, 0, 1], [0, 0, 0, 3, 0], [1, 1, 1, 0, 4]]
[[1, 1, 1, 0, 1], [1, 2, 2, 0, 1], [0, 0, 0, 3, 0], [1, 1, 1, 0, 4], [1, 2, 2, 0, 1]]


'aaba'